In [163]:
from pydub import AudioSegment,silence
from pydub.playback import play
import speech_recognition as sr
r = sr.Recognizer()

minSilenceLen = 1200
lengthAtLeast = 12 # in Seconds...
audiobookMp3Path = "/Users/joker/Jupyter Workspace/pydub/The Sovereign Individual/02. Preface.mp3"

def split(filepath):
    sound = AudioSegment.from_mp3(filepath)
    dBFS = sound.dBFS
    chunks = silence.split_on_silence(sound, 
        min_silence_len = minSilenceLen,
        silence_thresh = dBFS-16,
        keep_silence = True
    )
    return chunks

chunks = split(audiobookMp3Path)
# print(chunks)
print(len(chunks))

18


In [164]:
target_length = lengthAtLeast * 1000 #setting minimum length of each chunk to 25 seconds
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)

# print(output_chunks)
print(len(output_chunks))

12


In [165]:
for i, chunk in enumerate(output_chunks):
    j = f'{i+1}'
    chunk.export(f"{i+1}.wav".format("wav"), bitrate="48k", format="wav")
    # chunk.export(f"{i}.mp3".format("mp3"), bitrate="48k", format="mp3")

print("Done!")

In [166]:
wholetext=[]
for i in range(1, len(output_chunks)+1):
    with sr.AudioFile(f'{i}.wav') as source:
        # listen for the data (load audio to memory)
        audio_data = r.record(source)
        # recognize (convert from speech to text)
        text = r.recognize_google(audio_data)
        wholetext.append(text)
        print(f'\n{i}\n{text}\n')

print("\nDone!")


1
preface medieval men despaired of the will they thought of humans as wounded and weak but they respected the intellect


2
they thought even humans if we think carefully have the power to answer the most profound questions of God and the universe Modern Men worship the will but they despair of the intellect


3
the wisdom of crowds the swerve of random particles the influence of unconscious biases all of these contemporary cliches are ways to talk about intellectual weakness or ways to talk ourselves into it


4
Lord William rees-mogg and James Dale Davidson do not promise answers about God and the universe nor do they Supply any but their investigation of Mega politics and anatomy of the forces at work in history and a set of predictions for the near future is unusual or even countercultural because it applies human reason to matters that we have been taught to leave to chance or fate


5
looking back almost a quarter-century after the first publication of the sovereign individual 

In [167]:
html =[]

htmlhead="""
<!DOCTYPE html>
<html>
<style>
	body {
		padding: 50px 100px;
		font-family: 'Helvetica Neue UltraLight', Georgia, 'Times New Roman', Times, serif;
		font-size: 16px;
		line-height: 2.2em;
		background-color: #F1F3F4;
	}
	blockquote {
		border-left: 2px solid #ddd;
		padding-left: 25px;
	}
    em {
        font-color: #719f79;
    }
</style>
<body>
"""

htmlfooter="""
</body>
</html>
"""

html.append(htmlhead)

for i, text in enumerate(wholetext):
    text = f"""
<p><em>{i+1}</em>
<blockquote>
{text}
</blockquote>
</p>
<audio controls loop>
    <source src="{i+1}.wav"
    type="audio/wav">
</audio>
    """
    html.append(text)
    
html.append(htmlfooter)

htmlfile = open("02. Preface - audio-with-text.html", "w")
for line in html:
    htmlfile.write(line)
htmlfile.close()

print("Done!")


done!
